In [1]:
%run Reddit_import_and_functions.ipynb

pd.set_option('display.max_colwidth',350)

In [2]:
list_all_hdf5("masterdata.hdf5")

Train1Ytrain
Train1dset
Train2dset
X_Train2
columnsums
columnsums/columnindextodrop
columnsums/columnsumshalf0
columnsums/columnsumshalf1
environment
rowsums
rowsums/columnsumsCrypto
rowsums/columnsumsEnv
rowsums/columnsumsTrain2
rowsums/crypto_dset
rowsums/env_dset
rowsums/rowsumsCrypto
rowsums/rowsumsEnv
rowsums/rowsumsTrain2
rowsums/rowsumshalf0
rowsums/rowsumshalf1
y_train2


In [2]:
with h5py.File("masterdata.hdf5", "r") as z:
    dset = z["rowsums/env_dset"]
    X_train_env = dset[:]
    print(X_train_env.shape)
#     dset = z["rowsums/crypto_dset"]
#     X_train_crypto = dset[:]
#     print(X_train_crypto.shape)

(146787, 17847)


In [3]:
dfraw = pd.read_csv('Train1allinfo.csv')
dfraw.columns

Index(['Unnamed: 0', 'index', 'datetime', 'corpus', 'permalink', 'author',
       'subreddit'],
      dtype='object')

In [4]:
envmask = dfraw['subreddit'].astype(bool)
cryptomask = 1 - envmask
cryptomask = cryptomask.astype(bool)
print(cryptomask.sum())
envmask.sum()

293358


146787

In [5]:
df = dfraw[envmask]
dfraw = None

In [6]:
headers = pd.read_csv('masterFeaturesVector.csv')


In [7]:
X_train_env = pd.DataFrame(data=X_train_env, dtype='int16', 
                      index=df[envmask].index)
X_train_env.shape

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


(146787, 17847)

In [8]:
df[envmask]['datetime'].sort_values(ascending=False) # 1.545116e+09
df[envmask]['datetime'].sort_values() # 1.292656e+09
endtimestamp = 1.545116e+09
starttimestamp = 1.292656e+09

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [9]:
date_stamp5 = datetime.datetime.utcfromtimestamp(int(endtimestamp)).timestamp()

In [10]:
date_stamp0 = datetime.datetime.utcfromtimestamp(int(starttimestamp)).timestamp()
# year = date_tuple.year
# month = date_tuple.month
# day = date_tuple.day
# year, month, day
# hour = date_tuple.hour
# second = date_tuple.second

In [11]:
date_stamp1 = datetime.datetime((2010+2), 12, 18, 7, 6, 40).timestamp()
date_stamp2 = datetime.datetime((2010+2+1), 12, 18, 7, 6, 40).timestamp()
date_stamp3 = datetime.datetime((2010+2+1+2), 12, 18, 7, 6, 40).timestamp()
date_stamp4 = datetime.datetime((2010+2+1+2+1), 12, 18, 7, 6, 40).timestamp()

In [12]:
years1mask = (df['datetime'] > date_stamp0) & (df['datetime'] < date_stamp1)
years1mask.sum()

34905

In [13]:
years2mask = (df['datetime'] > date_stamp2) & (df['datetime'] < date_stamp3)
years2mask.sum()

37923

In [14]:
years3mask = (df['datetime'] > date_stamp4) & (df['datetime'] < date_stamp5)
years3mask.sum()

37558

In [15]:
with h5py.File("masterdata.hdf5",'a') as f :
    dset = f['columnsums/columnindextodrop']
    columnindextodrop = dset[:]

In [24]:
keepcols = set(range(17847)).symmetric_difference(columnindextodrop)
print(len(keepcols))
keepcols = list(keepcols)

10584


In [17]:
X_train_env.drop(columnindextodrop, axis=1, inplace=True)

In [18]:
X_train_env.loc[df[years1mask].index,'date_class'] = 0

In [19]:
X_train_env.loc[df[years2mask].index,'date_class'] = 1

In [20]:
X_train_env.loc[df[years3mask].index,'date_class'] = 2

In [21]:
X_train_env['date_class'].value_counts()

1.0    37923
2.0    37558
0.0    34905
Name: date_class, dtype: int64

In [22]:
X_train_env.shape

(146787, 10585)

In [26]:
X_train_env.dropna(axis=0, inplace=True)

In [27]:
X_train_env.shape

(110386, 10585)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_train_env.drop('date_class',axis=1), 
                                                    X_train_env['date_class'],train_size=0.5)

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [29]:
# all parameters will be kept at default except possible tweaking of alpha
nb = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
nbmodel = nb.fit(X_train, y_train)

In [32]:
pred1 = nbmodel.predict(X_train)
predtest = nbmodel.predict(X_test)

In [33]:
results = pd.DataFrame(index=X_train_env.index)

In [37]:
predfinal = pd.Series(data=pred1, index=X_train.index, name='nb3_pred')

In [ ]:
results['nb3_pred'] = predfinal

In [40]:
results['nb3_errors'] = results['nb3_pred'] - y_train

In [41]:
confusion_matrix(y_train, predfinal)

array([[12325,  3114,  1975],
       [ 3823, 11687,  3477],
       [ 2380,  3867, 12545]], dtype=int64)

In [42]:
y_train.shape

(55193,)

In [43]:
confusion_matrix(y_train, predfinal) / 55193

array([[0.2233073 , 0.0564202 , 0.03578352],
       [0.06926603, 0.21174787, 0.06299712],
       [0.04312141, 0.07006323, 0.22729332]])

In [44]:
confusion_matrix(y_test, predtest)

array([[11102,  3886,  2503],
       [ 4468, 10206,  4262],
       [ 2688,  4587, 11491]], dtype=int64)

In [45]:
y_test.shape

(55193,)

In [46]:
confusion_matrix(y_test, predtest) / 55193

array([[0.2011487 , 0.07040748, 0.04534995],
       [0.08095229, 0.18491475, 0.07721994],
       [0.04870183, 0.08310837, 0.20819669]])